In [1]:
import asyncio
import json
from utils import llm_call, llm_call_async

In [4]:
# 병렬 처리를 위한 함수
async def run_llm_parallel(prompt_list):
    tasks = [llm_call_async(prompt) for prompt in prompt_list]
    responses = []
    
    for task in asyncio.as_completed(tasks):
        result = await task
        responses.append(result)
    return responses

In [5]:
def get_orchestrator_prompt(user_query):
    return f"""
다음 사용자 질문을 분석하고, 이를 3개의 관련된 하위 질문으로 분해하십시오:

다음 형식으로 응답을 제공하십시오:

{{
    "analysis": "사용자 질문에 대한 이해를 상세히 설명하고, 작성한 하위 질문들의 근거를 설명하십시오.",
    "subtasks": [
        {{
            "description": "이 하위 질문의 초점과 의도를 설명하십시오.",
            "sub_question": "질문 1"
        }},
        {{
            "description": "이 하위 질문의 초점과 의도를 설명하십시오.",
            "sub_question": "질문 2"
        }}
        // 필요에 따라 추가 하위 질문 포함
    ]
}}
최대 3개의 하위 질문을 생성하세요

사용자 질문: {user_query}

"""

In [6]:
def get_worker_prompt(user_query, sub_question, description):
    return f"""
    다음 사용자 질문에서 파생된 하위 질문을 다루는 작업을 맡았습니다:
    원래 질문:  {user_query}
    하위 질문: {sub_question}

    지침: {description}

    하위 질문을 철저히 다루는 포괄적이고 상세한 응답을 해주세요
    """

In [10]:
async def orchestrate_task(user_query):
    """
    오케스트레이터를 실행하여 원래 질문을 하위 질문으로 분해하고,
    각각의 하위 질문을 병렬적으로 실행하여 종합적인 응답을 생성합니다.
    """
    
    # 1단계 : 사용자 질문 기반으로 여러 질문 도출
    orchestrator_prompt = get_orchestrator_prompt(user_query)
    print("\n============================orchestrator prompt============================\n")
    print(orchestrator_prompt)
    orchecstrator_response = llm_call(orchestrator_prompt, model="gemini-1.5-flash-8b")
    
    # 응답 결과 (1단계) 출력
    print("\n============================orchestrator response============================\n")
    print(orchecstrator_response)
    
    response_json = json.loads(orchecstrator_response.replace('```json','').replace('```',''))
    
    analysis = response_json.get("analysis", "")
    sub_tasks = response_json.get("subtasks", [])
    
    # 2단계 : 각 하위질문에 대한 LLM 호출
    worker_prompts = [get_worker_prompt(user_query, task["sub_question"], task["description"]) for task in sub_tasks]
    print("\n============================worker prompts============================\n")
    for prompt in worker_prompts:
        print(prompt)
        
    worker_responses = await run_llm_parallel(worker_prompts)
    
    # 응답결과(2단계) 출력
    print("\n============================worker responses============================\n")
    for response in worker_responses:
        print(response)
        
    # 3단계 : 하위질문 응답 종합 및 LLM 호출
    aggregator_prompt = f"""아래는 사용자의 원래 질문에 대해서 하위 질문을 나누고 응답한 결과입니다.
    아래 질문 및 응답내용을 포함한 최종 응답을 제공해주세요.
    ## 요청사항
    - 하위질문 응답내용이 최대한 포괄적이고 상세하게 포함되어야 합니다
    사용자의 원래 질문:
    {user_query}

    하위 질문 및 응답:
    """
    
    for i in range(len(sub_tasks)):
        aggregator_prompt += f"\n{i+1}. 하위 질문: {sub_tasks[i]['sub_question']}\n"
        aggregator_prompt += f"\n   응답: {worker_responses[i]}\n"
        
    print("\n============================aggregator prompt============================\n")
    print(aggregator_prompt)
    
    final_response = llm_call(aggregator_prompt, model="gemini-1.5-flash-8b")
    
    return final_response


In [11]:
import time

async def main():
    user_query = "AI는 미래 일자리에 어떤 영향을 미칠까?"
    
    # CASE 1 : 그냥 질문 했을 때
    print("\n============================CASE 1 : 그냥 질문 했을 때============================\n")
    print(llm_call(user_query, model="gemini-1.5-pro-latest"))
    
    time.sleep(5)
    
    # CASE 2 : 오케스트레이터 실행
    print("\n============================CASE 2 : 오케스트레이터 실행============================\n")
    final_output = await orchestrate_task(user_query)
    print("\n============================final output============================\n")
    print(final_output)
    
await main()


============================CASE 1 : 그냥 질문 했을 때============================

 AI는 미래 일자리에 상당한 영향을 미칠 것으로 예상되며, 그 영향은 다면적이고 복잡합니다. 단순히 일자리를 "뺏는다"는 관점보다는 변화시키고 새로운 기회를 창출한다는 관점으로 접근하는 것이 중요합니다.

**1. 자동화로 인한 일자리 감소:**

* **반복적이고 예측 가능한 업무:** 제조업, 물류, 고객 서비스, 데이터 입력 등 단순 반복적인 업무는 AI와 로봇에 의해 자동화될 가능성이 높습니다.
* **일부 전문직:**  의료 진단, 법률 문서 검토, 회계 감사 등 일부 전문직 영역에서도 AI가 인간의 역할을 대체할 수 있는 부분이 존재합니다.

**2. 새로운 일자리 창출:**

* **AI 관련 직종:** AI 개발자, 데이터 과학자, AI 트레이너, AI 윤리 전문가 등 AI 기술 자체와 관련된 새로운 직업이 등장하고 수요가 증가할 것입니다.
* **AI 보완 직종:** AI는 인간의 능력을 보완하고 향상시키는 도구로 활용될 수 있습니다. 의료, 교육, 금융 등 다양한 분야에서 AI를 활용하여 업무 효율성을 높이고 새로운 서비스를 제공하는 직업이 생겨날 것입니다.
* **창의성 및 감성 기반 직종:** 예술가, 디자이너, 작가, 상담사, 사회복지사 등 인간의 창의성, 감성, 공감 능력이 중요한 직업은 AI로 대체되기 어려울 것으로 예상되며, 오히려 수요가 증가할 수 있습니다.

**3. 기존 일자리의 변화:**

* **업무 스킬 변화:** 많은 직업에서 AI 활용 능력이 필수적인 업무 스킬로 자리 잡을 것입니다.  AI 도구를 사용하고 데이터를 분석하는 능력이 중요해질 것입니다.
* **인간과 AI의 협업:**  AI는 인간의 업무를 완전히 대체하기보다는 보조하는 역할을 수행할 가능성이 높습니다. 인간과 AI가 협력하여 업무 효율성을 높이는 방식으로 일자리가 변화할 것입니다.


**4. 사회적 영향:**
